<a href="http://landlab.github.io"><img style="float: left" src="https://raw.githubusercontent.com/landlab/tutorials/master/landlab_header.png"></a>

# Using Landlab to explore a diffusive hillslope in the piedmont of North Carolina 

This notebook was developed in collaboration with Karl Wegmann at North Carolina State University. 

<hr>
For instructions on how to run an interactive IPython notebook, click here: <a href="https://github.com/landlab/tutorials/blob/master/README.md">https://github.com/landlab/tutorials/blob/master/README.md</a><br>
For tutorials on learning Landlab, click here: <a href="https://github.com/landlab/landlab/wiki/Tutorials">https://github.com/landlab/landlab/wiki/Tutorials</a>
<hr>


**What is this notebook?**

This notebook is designed to be an exercise for students in a quantative geomorphology class. This notebook is meant to illustrate to students how a model can be used to understand and interpret real landscapes. This notebook is not designed to teach students how to code, nor is it designed to teach students how to use Landlab.

This notebook provides data from a real landscape in North Carolina in order to compare the shape of this hillslope with those produced by models using linear diffusion.

This notebook steps students through exercises to quantify the diffusivity on this hillslope, assuming that sediment transport on this hillslope follows a linear diffusion law. 

**Application of linear diffusion transport law:**

The tranport law applied here is of the form:
\begin{equation}
q_s = -D \nabla z
\end{equation}
where ${q}_s$ is the transport rate with dimensions of L$^2$T$^{-1}$;  $D$ is a transport coefficient with dimensions of L$^2$T$^{-1}$; and $z$ is elevation. $\nabla z$ is the gradient in the landscape, and landscape slope is $-\nabla z$, hence the negative in the above equation. 
 
Changes in elevation, or erosion, are calculated from conservation of mass:
\begin{equation}
\frac{dz}{dt} = U-\nabla q_s
\end{equation}
where $U$ is the rock uplift rate, with dimensions LT$^{-1}$ and $t$ is time.

Note that at steady state the rock uplift rate sets the erosion rate. If the erosion rate in a landscape is a known, a modeling scenario in which the landscape is uplifting at the same rate as the known erosion rate will eventually lead to a landscape that is also eroding at that rate. 

**What will we do with Landlab?**

Here we will use the Landlab component *LinearDiffuser*, to explore whether hillslopes evolved according to a linear diffusion rule look like the example North Carolina hillslope. We will just be eye-balling similarities. No rigorous comparison between the real and modeled hillslope is performed.

More general background on applying hillslope process models to real landscapescan be found in this paper:

Roering, Joshua J. (2008) "How well can hillslope evolution models “explain” topography? Simulating soil transport and production with high-resolution topographic data." Geological Society of America Bulletin.

This paper has a nice compilation of measured landscape diffusivity values:

Perron, J. T., (2017) "Climate and the pace of erosional landscape evolution" Annual Review of Earth and Planetary Sciences.

**What do you need to know about this Landscape?**

General information on this landscape can be found in this reference:

Wegmann, K. W., Lewis, R. Q., & Hunt, M. C. (2012). "Historic mill ponds and piedmont stream water quality: Making the connection near Raleigh, North Carolina". Field Guides.

For now the following information is provided.

- The site is in the Piedmont near Raleigh, North Carolina.
- The site is in the W.B. Umstead State Park which is in the Sycamore Creek watershed. Sycamore Creek drains into the Neuse River.
- The area gets ~ 1.25 m/yr of rainfall.
- The site is currently forested.
- Basin average erosion rates from the area are on the order of 10 m/million years, or 1e-5 m/yr. These rates are averaged over millenial time scales. These rates are from a nearby area similar to this, but not exactly this study area.
- The site has been heavily impacted from agriculture due to European settlers, beginning in the 1600s. 

**What you need to do:**

You will start out by making the assumption that the landscape is eroding at a rate of 1e-5 m/yr. 

As you will see, the study area hillslope profile has a form similar to what one would expect from a uniformly eroding diffusive profile. **Your first goal is to find the diffusivity and time required for the profile to reach steady state given the background erosion rate of 1e-5 m/yr.** You can do that by running the code below. **Take special note of Code Block 6 where you set the diffusivity. Make sure choose a resonable initial value, and record what your initial value was.** Also be prepared to change model run time (Code Block 7) for your initial experiments. Once you iterate to a modeled profile that you feel reasonably matches the DEM profile (just eye-ball the similarity), you will be ready to answer the questions that follow all of the code. 

**How to run a Jupyter notebook:**

Read the text boxes and sequentially run each code block (shift - enter OR got to the _Cell_ pulldown menu at the top and choose _Run Cells_). 

Remember that you can always go to the _Kernel_ pulldown menu at the top and choose _Restart & Clear Output_ or _Restart & Run All_ if you change things and want to start afresh. If you just change one code block and rerun only that code block, only the parts of the code in that code block will be updated. (E.g. if you change parameters but don't reset the code blocks that initialize run time or topography, then these values will not be reset.) 

**Now on to the code...**

We start by importing libraries that we will need to run the code. You should not need to edit this code block.

In [ ]:
# Code Block 1

import numpy as np
from landlab.io import read_esri_ascii
from landlab.plot.imshow import imshow_grid
import matplotlib.pyplot as plt
#below is to make plots show up in the notebook
%matplotlib inline 

Below is a helper function for this notebook. You should not need to edit this code block.

In [ ]:
# Code Block 2

class FieldProfiler:

    def __init__(self, mg, field, start, end):
                
        self._grid = mg
        self._field = field
        self.p0 = start
        self.p1 = end
        
        # Get coordinates
        x0,y0 = self.p0[0], self.p0[1]
        x1,y1 = self.p1[0], self.p1[1]
        length = int(np.hypot(x1 - x0, y1 - y0)) / mg.dx
        (x, y) = np.linspace(x0, x1, length), np.linspace(y0, y1, length)
        
        self.x = np.append(0, (np.cumsum(np.diff(x))))

        # Extract the values along the line.
        self.ids = []
        self.z = []
        for i,xi in enumerate(x):
            node = self._grid.find_nearest_node((xi, y[i]))
            zi = self._grid.at_node[self._field][node]
            
            self.ids.append(node)
            self.z.append(zi)

    def plot(self):
        
        self.fig = plt.subplots(nrows=2, ncols=1)
        
        plt.subplot(2, 1, 1)
        imshow_grid(self._grid, self._field)
        plt.plot([self.p0[0], self.p1[0]], [self.p0[1], self.p1[1]], 'co-')
        
        plt.subplot(2,1,2)
        plt.plot(self.x, self.z, 'k')
        plt.xlabel('Distance')
        plt.ylabel('Elevation')
        
        plt.tight_layout()

Now we import the DEM from our example landscape, set boundary conditions, and plot the hillslope profile that we will compare model outcome with. Note that the location of the hillslope profile has already been chosen for you, but you could easily change this if you want to explore further. For the rest of this notebook we will focus on the western side of the slope, which is more easily explained using a linear diffusion equation.

In [ ]:
# Code Block 3

# Import DEM, note this is units of meters
(nc_grid, z) = read_esri_ascii('ascii_dem.txt', name='topographic__elevation')
# set boundary conditions on DEM for plotting reasons
nc_grid.set_nodata_nodes_to_closed(z,-9999)

# Create profile (p) between start and end coordinates
p_start = (54, 194) # (x,y) of the starting point of profile
p_end = (282, 109) # (x,y) of the end point of profile
# Calculate the profile
p = FieldProfiler(nc_grid, 'topographic__elevation', p_start, p_end)
# Make a plot of the profile
p.plot()
# Get distance and elevation data along the profile line
field_dist = p.x # distance along the profile
field_z = p.z # elevation along the profile

We will create a grid for our model using Landlab's *RasterModelGrid* class, which we need to import. We make a grid with *dx* = *dy* = 6.0960122 m (same resolution as the imported DEM). We make a grid that has 5 columns and 38 rows, to match the length of the profile from the real landscape.

We set the initial elevation of the topography to be 87 m everywhere. We also set boundary conditions. (Sediment can leave the top and bottom of the grid). 

In [ ]:
# Code Block 4

from landlab import RasterModelGrid
mg = RasterModelGrid((38, 5), 6.0960122) # make grid
z_vals = mg.add_ones('topographic__elevation', at='node') # initialize z values
initial_elevation = np.multiply(z_vals, 86.) # this line and next set elevation to 87. m
z_vals += initial_elevation

mg.set_closed_boundaries_at_grid_edges(True, False, True, False)

Below we initialize values for plotting variables down the middle of the hillslope.

In [ ]:
# Code Block 5

ycoord_rast = mg.node_vector_to_raster(mg.node_y)
ys_grid = ycoord_rast[:, 2]

Now we import and initialize the *LinearDiffuser* component. In this case the units on our diffusivity coefficient, or transport coefficient, are m$^2$yr$^{-1}$.

**NOTE to Students:** You need to chose a reasonable initial value for D (diffusivity). Remember you need to justify your initial guess for D.

In [ ]:
# Code Block 6

from landlab.components import LinearDiffuser
D = 1 # value in m^2/yr
lin_diffuse = LinearDiffuser(mg, linear_diffusivity=D)

We now initialize a few more parameters.

In [ ]:
# Code Block 7

# Uniform rate of rock uplift, which drives uniform erosion at steady state
uplift_rate = 0.00001  # m/year

# Total time in years that the model will run for.
runtime = 10000 # years

# Stability criteria for timestep dt.  Coefficient can be changed
# depending on our tolerance for stability vs tolerance for run time.
# Do not change this.
dt = 0.5 * mg.dx * mg.dx / D

# nt is number of time steps
nt = int(runtime // dt)

# Below is to keep track of time for labeling plots
time_counter = 0

# length of uplift over a single time step, meters
uplift_per_step = uplift_rate * dt

Now we figure out the analytical solution for the elevation of the steady-state profile.

In [ ]:
# Code Block 8

ys = np.arange(mg.number_of_node_rows*mg.dx-mg.dx)

# location of divide or ridge crest -> middle of grid 
# based on boundary conds.
divide_loc = (mg.number_of_node_rows*mg.dx-mg.dx)/2

# half-width of the ridge
half_width = (mg.number_of_node_rows*mg.dx-mg.dx)/2

# analytical solution for elevation under linear diffusion at steady state
zs = np.add( (uplift_rate/(2*D)) * \
    (np.power(half_width, 2) - np.power(ys - divide_loc, 2)), 87)

Before we evolve the landscape, let's look at the model initial topography, the analytical solution, and the field profile.

In [ ]:
# Code Block 9

plt.figure()
imshow_grid(mg, 'topographic__elevation')
plt.title('initial topography')
plt.figure()
elev_rast = mg.node_vector_to_raster(
            mg.at_node['topographic__elevation'])
plt.plot(ys_grid, elev_rast[:, 2], 'r-', label='model')
plt.plot(ys, zs, 'k--', label='analytical solution')
plt.plot(field_dist, field_z, 'b:', label='field data')
plt.xlabel('horizontal distance (m)')
plt.ylabel('vertical elevation (m)')
plt.legend(loc='lower center')
plt.title('before running model')

Now we are ready to evolve the landscape and compare it to the steady state solution.

Below is the time loop that does all the calculations. 

In [ ]:
# Code Block 10

for i in range(nt):
    mg['node']['topographic__elevation'][mg.core_nodes] += uplift_per_step
    lin_diffuse.run_one_step(dt)
    time_counter += dt

Now we plot the evolved cross-section.

In [ ]:
# Code Block 11

elev_rast = mg.node_vector_to_raster(
            mg.at_node['topographic__elevation'])
plt.plot(ys_grid, elev_rast[:, 2], 'r-', label='model')
plt.plot(ys, zs, 'k--', label='analytical solution')
plt.plot(field_dist, field_z, 'b:', label='field data')
plt.xlabel('horizontal distance (m)')
plt.ylabel('vertical elevation (m)')
plt.legend(loc='lower center')
plt.title('topographic cross section at time %s, with D = %s m^2/yr'%(time_counter,D))

**Questions to answer and further model experiments:**

_NOTE_ You should hand in a typed, electronic document that has figures embedded in the document. The document can be any type of readable file. Please include your name in the file name that you turn in.

**Questions:**
1. Your first task was to find the diffusivity and time to steady state required for the model to produce a landscape similar to the field area near Raleigh, NC. (Time to steady state can be approximate, you don't have to hone in on the exact year. I am just looking for order of magnitude time.) What was your initial guess for a diffusivity? What information did you use to choose that initial value? What was the value that produced a good match between the modeled and real hillslope form? (Again, you can eye-ball the match. Just get close visually.) Compared with other values of diffusivity that have been reported in the literature, how does your value compare? That is, given what you know about this landscape, does this seem like a reasonable diffusivity? Explain.
2. What was the estimated time to steady state using the model? Compare the time to steady state with the time over which erosion rates were measured and the time period over which humans have dramatically impacted this landscape. What are the implications of the differences among the time to steady state, the time over which erosion rates were measured, and time over which humans have impacted this system?
3. It is possible that humans accelerated erosion rates (from the background, pre-settlement rate, which here we can assume is set from "background rock uplift") by almost 500 times. If that happened, how would the profile of today be different from the pre-settlement profile (~400 years ago)? Does this have any implications for your estimated diffusivity? What would you need to change in the model to "simulate" this? Develop a scenario in which you evolve a landscape to steady state, then you change some parameters, and then evolve the landscape another 400 years to simulate the impact of humans. Provide plots and describe in detail exactly what you did to simulate the pre- and post-human landscape. Are there noticeable changes in the landscape? 